# Decision analysis with Julia `Agents.jl` using Petitti, 2000, 2e. - Chapter 2 
Tomás Aragón, Updated 2026-01-21

## Backgroud

This Julia Jupyter Notebook demonstrates decision analysis using agent-based (ABM) modeling with the `Agents.jl` Julia package. The example is based on the decision tree from Diana B. Petitti. *Meta-Analysis, Decision Analysis, and Cost-Effectiveness Analysis: Methods for Quantitative Synthesis in Medicine*. 2nd ed. Monographs in Epidemiology and Biostatistics, v. 31. Oxford University Press, 2000. https://doi.org/10.1093/acprof:oso/9780195133646.001.0001.

The ABM replicates these previous decision analyses:
- DA using a decision tree (`rdecision`): [R Jupyter Notebook (Part 2)](https://github.com/tomasaragon/di4h/blob/main/nb/Petitti2000/NB_R_rdecision_part2.ipynb)   
- DA using influence diagrams (`DecisionProgramming.jl`): [Julia Jupyter Notebook](https://github.com/tomasaragon/di4h/blob/main/nb/Petitti2000/NB_Julia_DecisionProgramming-jl.ipynb)

The `Agents.jl` package documentation is here: https://juliadynamics.github.io/Agents.jl/stable/.

To learn more about the Julia language visit: https://julialang.org .

Here is the decision tree from Petitti, 2000, 2e, Chapter 2, Figure 2-6 (FIGURE 1). 

<figure>
<img src="Petitti_2000_2e_ch02_fig2-6.png" width="800" alt="Petitti 2000 Figure 2-6"/>
<figcaption>FIGURE 1. Decision tree from Petitti, 2000, 2e, Chapter 2, Figure 2-6.</figcaption>
</figure>

Now, here is the decision tree as an influence diagram (FIGURE 2). Notice there is no arrow from the measles node to the value node. The value node only contains input information on revaccination status and mortality based on the scenario. If measles morbidity had been included in the scenario then we would have added an arrow from the measles node to value node. I find influence diagrams more concise and interpretable. Because influence diagrams are directed acyclic graphs (DAGs), they promote causal reasoning. 

<figure>
<img src="img_measles_decision_network_drawio.png" width="600" alt="Petitti 2000 Figure 2-1"/>
<figcaption>FIGURE 2: Influence diagram for Figure 2-6 from Petitti, 2000, 2e, Chapter 2.</figcaption> 
</figure>

We have a human agent that can experience these states: revaccination, exposure to measles, measles infection, and death from measles.  
- $P(E = 1) = 0.20$
- $P(M = 1 \mid E = 1, R = 1) = 0.05$
- $P(M = 1 \mid E = 1, R = 0) = 0.33$
- $P(D = 1 \mid M = 1) = 0.0023$

where: $E$ is exposure to measles $\{1=yes, 0=no\}$, $M$ is measles infection $\{1=yes, 0=no\}$, $D$ is death from measles $\{1=yes, 0=no\}$, and $R$ is revaccination status $\{1=yes, 0=no\}$.

Steps to implement the agent-based model: 
1. create agent type
2. create agent stepping function
3. intitialize model: create model and add agents
4. run simulation
5. analyze results

## Julia code using Agents.jl

### Julia housekeeping
- activate environment
- ensure latest version of Agents.jl

In [11]:
#import Pkg # Run to update Agents.jl to v6+
## --- 1. Activate a new folder for this project
#Pkg.activate("DAProject");
## --- 2. Force install the latest version
#Pkg.update();
## --- 3. Check the version (should be v6+)
#Pkg.status("Agents");

### Load packages and set constants

In [12]:
using Agents
using Random
using Statistics

# Probabilities
const p_E1 = 0.20          
const p_M1_E1_R1 = 0.05    
const p_M1_E1_R0 = 0.33    
const p_D1_M1 = 0.0023     

# Utilities
const U_Alive = 1.0
const U_Dead = 0.0

0.0

### Step 1 - Create agent type
Agents.jl provides the `@agent` macro that enables us to create an agent type quickly with minimal coding. It takes this basic informatioin and constructs a new data type with a variable for tracking unique IDs, etc. In this simulation we will create 1 million agents. Each agents has five mutuable attributes: revaccination, exposure, measles, death, and utility. Revaccination status will be set by the analyst (me), the others are stochastic. 

In [13]:
@agent struct Patient(NoSpaceAgent)
    revaccinate::Bool 
    exposed::Bool     
    measles::Bool     
    dead::Bool        
    utility::Float64  
end

### Step 2 - Create agent stepping function
Next, we need to create a function that captures the possible state changes and transition probabilities of the an agent. Fortunately, this is a simple example: revaccination, exposure, measles, and death.

In [14]:
function agent_step!(agent::Patient, model)
    # 1. Chance Node E: Exposure
    # Use abmrng(model) to access the random number generator
    rng = abmrng(model)
    
    if rand(rng) < p_E1
        agent.exposed = true
        
        # 2. Chance Node M: Measles
        prob_measles = agent.revaccinate ? p_M1_E1_R1 : p_M1_E1_R0
        
        if rand(rng) < prob_measles
            agent.measles = true
            
            # 3. Chance Node D: Death
            if rand(rng) < p_D1_M1
                agent.dead = true
                agent.utility = U_Dead
            else
                agent.dead = false
                agent.utility = U_Alive
            end
        else
            agent.measles = false
            agent.utility = U_Alive
        end
    else
        agent.exposed = false
        agent.utility = U_Alive
    end
end

agent_step! (generic function with 1 method)

### Step 3 - Intitialize model: create model and add agents
At this step we create the model using the `StandardABM` function which, for our inputs, has 4 arguments:
`StandardABM(::Type{A}, space::S; rng, agent_step!, ...)`. The arguments before the `;` are positional arguments and those after the `;` are keyword arguments. The first argument is the agent type we created in Step 1. The second argument is the space type. In this case we use `NoSpace()` because we do not need a spatial component. The third argument is the random number generator (RNG). If we do not provide one, a default RNG will be used. The fourth argument is the agent stepping function we created in Step 2. Use `methods(StandardABM)` to see other mthods for the `StandardABM` function.    

Next, we add agents to the model using the `add_agent!` function. Arguments 2 to 6 are the agent attributes we created in Step 1. revaccinate, exposed, measles, dead, and utility. We will create 1 million agents with revaccination status set to true or false based on the scenario. The other attributes will be set to false or 0 initially.

In [15]:
function initialize_model(; n_agents = 100_000, decision_revaccinate = true)
    space = nothing
    # Create model with v6 syntax
    model = StandardABM(Patient, space; rng = MersenneTwister(42), agent_step! = agent_step!)
    
    # Create agents
    for i in 1:n_agents
        add_agent!(model, decision_revaccinate, false, false, false, 0.0)
    end
    
    return model
end

initialize_model (generic function with 1 method)

### Step 4 - Run simulation

In [16]:
# --- Scenario 1: R1 (Revaccinate) ---
model_R1 = initialize_model(n_agents = 1_000_000, decision_revaccinate = true)
run!(model_R1, 1) 

eu_R1 = mean(model_R1[id].utility for id in allids(model_R1))

# --- Scenario 2: R0 (Do Not Revaccinate) ---
model_R0 = initialize_model(n_agents = 1_000_000, decision_revaccinate = false)
run!(model_R0, 1) 

eu_R0 = mean(model_R0[id].utility for id in allids(model_R0))

# --- Results ---
println("--- Agents.jl Monte Carlo Results ---")
println("R1 (Revaccinate) Expected Utility: $eu_R1")
println("R0 (No Revax)    Expected Utility: $eu_R0")

--- Agents.jl Monte Carlo Results ---
R1 (Revaccinate) Expected Utility: 0.999968
R0 (No Revax)    Expected Utility: 0.999843


### Step 5 - Analyze results

In [17]:
# calculate risk difference as on pp. 27-28
risk_death_no_revax = 1 - eu_R0
risk_death_revax = 1 - eu_R1
risk_difference = risk_death_no_revax - risk_death_revax
round(risk_difference, digits = 6)


0.000125

From Petitti, pp. 27-28: "The difference in the expected probability of death from measles between a strategy of revaccination and a strategy of no-revaccination is `0.000125`. This is interpreted to mean that 12.9 deaths from measles are prevented per 100,000 children revaccinated.

## Appendix - Selected Julia resources
- [The Julia Programming Language](https://julialang.org/) 
- [My journey from R to Julia: A very brief introduction to Julia for epidemiologists](https://tomasaragon.github.io/posts/2023-01-14-my-journey-from-r-to-julia/) (blog) 
- [Julia for Data Analysis: A book review for population health data scientists](https://tomasaragon.github.io/posts/2023-02-20-julia-for-data-analysis/) (blog)
